In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Sequential
from tensorflow.python.client import device_lib

In [35]:
image_size = (48, 48)
batch_size = 64

train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
    "data/data/images/train",
    label_mode='categorical',
    color_mode="grayscale",
    validation_split=0.2,
    subset="both",
    shuffle=True,
    seed=666,
    image_size=image_size,
    batch_size=batch_size,
)

Found 28821 files belonging to 7 classes.
Using 23057 files for training.
Using 5764 files for validation.


In [36]:
df_test = pd.read_csv("test_set.csv")

In [37]:
import cv2
import os

In [38]:
df_test

,id_img
0,10052
1,10065
2,10079
3,10095
4,10121
...,...
7061,9806
7062,9830
7063,9853
7064,9878


In [39]:
image_path = os.getcwd() + "\\data\\data\\images\\test"

In [40]:
image_path

'c:\\Users\\Usuario\\Desktop\\Proyecto CNN alegres\\red-neuronal-alegre\\data\\data\\images\\test'

In [41]:
imagenes = {"imagen" : [], "path" : []}
for file in os.listdir(image_path):
    file2 = file.split(".")
    imagenes["imagen"].append(file2[0])
    imagenes["path"].append(os.path.join(image_path, file))


In [42]:
df_test = pd.DataFrame(imagenes, columns=imagenes.keys())

In [43]:
df_test

,imagen,path
0,10004,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...
1,10019,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...
2,10023,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...
3,10029,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...
4,1003,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...
...,...,...
7061,9977,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...
7062,9979,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...
7063,9982,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...
7064,9984,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...


In [44]:
def cargar_y_convertir_imagen(ruta_imagen):
    image = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
    image_array = np.array(image)
    return image_array

df_test['imagenes'] =df_test['path'].apply(cargar_y_convertir_imagen)

In [45]:
df_test

,imagen,path,imagenes
0,10004,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...,"[[1, 1, 1, 2, 7, 15, 16, 9, 11, 14, 13, 14, 31..."
1,10019,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...,"[[159, 157, 158, 166, 170, 170, 172, 171, 166,..."
2,10023,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...,"[[195, 198, 193, 190, 140, 70, 54, 88, 169, 21..."
3,10029,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...,"[[41, 48, 57, 66, 68, 67, 75, 85, 94, 105, 109..."
4,1003,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...,"[[44, 33, 21, 23, 29, 29, 34, 37, 40, 40, 45, ..."
...,...,...,...
7061,9977,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...,"[[178, 139, 130, 131, 159, 188, 183, 239, 249,..."
7062,9979,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...,"[[243, 243, 238, 196, 185, 176, 168, 162, 157,..."
7063,9982,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...,"[[203, 229, 234, 238, 233, 232, 238, 235, 229,..."
7064,9984,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...,"[[57, 56, 57, 63, 61, 56, 58, 58, 61, 72, 69, ..."


In [46]:
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model, Sequential
from keras.optimizers import Adam

# number of possible label values
nb_classes = 7

# Initialising the CNN
model = Sequential()

# 1 - Convolution
model.add(Conv2D(64,(3,3), padding='same', input_shape=(48, 48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 2nd Convolution layer
model.add(Conv2D(128,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 3rd Convolution layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 4th Convolution layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flattening
model.add(Flatten())

# Fully connected layer 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(nb_classes, activation='softmax'))

model.compile( loss='categorical_crossentropy', metrics=['accuracy'])

In [47]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 48, 48, 64)        640       
                                                                 
 batch_normalization_18 (Ba  (None, 48, 48, 64)        256       
 tchNormalization)                                               
                                                                 
 activation_18 (Activation)  (None, 48, 48, 64)        0         
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 24, 24, 64)        0         
 ng2D)                                                           
                                                                 
 dropout_18 (Dropout)        (None, 24, 24, 64)        0         
                                                                 
 conv2d_13 (Conv2D)          (None, 24, 24, 128)      

In [48]:
epochs = 15

callbacks = [
    keras.callbacks.ModelCheckpoint("models/save_at_{epoch}.keras"),
    keras.callbacks.EarlyStopping(patience=5)
]
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
history = model.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=val_ds,
)

Epoch 1/15
361/361 [==============================] - 133s 359ms/step - loss: 1.7808 - accuracy: 0.3183 - val_loss: 1.5654 - val_accuracy: 0.3843
Epoch 2/15
361/361 [==============================] - 139s 385ms/step - loss: 1.4356 - accuracy: 0.4447 - val_loss: 1.5009 - val_accuracy: 0.4013
Epoch 3/15
361/361 [==============================] - 135s 374ms/step - loss: 1.2861 - accuracy: 0.5055 - val_loss: 1.3664 - val_accuracy: 0.4981
Epoch 4/15
361/361 [==============================] - 133s 367ms/step - loss: 1.2027 - accuracy: 0.5387 - val_loss: 1.3440 - val_accuracy: 0.4842
Epoch 5/15
361/361 [==============================] - 139s 384ms/step - loss: 1.1405 - accuracy: 0.5651 - val_loss: 1.2370 - val_accuracy: 0.5220
Epoch 6/15
361/361 [==============================] - 139s 386ms/step - loss: 1.0855 - accuracy: 0.5869 - val_loss: 1.3269 - val_accuracy: 0.4781
Epoch 7/15
361/361 [==============================] - 138s 382ms/step - loss: 1.0348 - accuracy: 0.6043 - val_loss: 1.3743 -

In [ ]:
from tensorflow.keras.models import load_model

best_model = None
best_val_accuracy = 0

for epoch in range(1, 16):
    model = load_model(f"models/save_at_{epoch}.keras")
    loss, accuracy = model.evaluate(val_ds)
    if accuracy > best_val_accuracy:
        best_val_accuracy = accuracy
        best_model = model

print("El mejor modelo es:", best_model)

In [ ]:

def plot_history_loss(history):
    tr_acc = history.history['accuracy']
    tr_loss = history.history['loss']
    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']
    index_loss = np.argmin(val_loss)
    val_lowest = val_loss[index_loss]
    index_acc = np.argmax(val_acc)
    acc_highest = val_acc[index_acc]
    Epochs = [i+1 for i in range(len(tr_acc))]
    loss_label = f'best epoch= {str(index_loss + 1)}'
    acc_label = f'best epoch= {str(index_acc + 1)}'
    # Plot training history
    plt.figure(figsize= (20, 8))
    plt.style.use('fivethirtyeight')
    plt.subplot(1, 2, 1)
    plt.plot(Epochs, tr_loss, 'r', label= 'Training loss')
    plt.plot(Epochs, val_loss, 'g', label= 'Validation loss')
    plt.scatter(index_loss + 1, val_lowest, s= 150, c= 'blue', label= loss_label)
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(Epochs, tr_acc, 'r', label= 'Training Accuracy')
    plt.plot(Epochs, val_acc, 'g', label= 'Validation Accuracy')
    plt.scatter(index_acc + 1 , acc_highest, s= 150, c= 'blue', label= acc_label)
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout
    plt.show()


In [50]:
TEST_IMAGE_PATH = os.getcwd() + '/data/data/images/test'

In [51]:
list_img = []

In [52]:
for file in os.listdir(TEST_IMAGE_PATH):
    try:
        file_path = os.path.join(TEST_IMAGE_PATH, file)
        img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        list_img.append(img.flatten())
    except Exception as e:
        print("Error: ",e)

In [53]:
list_img[0].shape

(2304,)

In [ ]:
plot_history_loss(history)

In [55]:
from tensorflow.keras.models import load_model

final_modal = load_model("models/save_at_10.keras")

In [65]:
X_test = np.stack(df_test['imagenes'].tolist())
X_test = np.expand_dims(X_test, axis=-1)
#X_test = np.repeat(X_test, 3, axis=-1)

In [66]:
X_test.shape

(7066, 48, 48, 1)

In [67]:
X_test = X_test / 255.0

In [68]:
class_names = train_ds.class_names
class_names

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [69]:
y_pred = final_modal.predict(X_test)
y_pred


221/221 [==============================] - 11s 47ms/step


array([[0.1425497 , 0.00091945, 0.04819369, ..., 0.26819044, 0.406098  ,
        0.04182393],
       [0.14250109, 0.00092515, 0.04818733, ..., 0.26786086, 0.40620732,
        0.04166403],
       [0.1425876 , 0.00092523, 0.04815482, ..., 0.26641604, 0.4070518 ,
        0.04180638],
       ...,
       [0.14227545, 0.00092387, 0.04808681, ..., 0.26762414, 0.40662014,
        0.04190677],
       [0.14276549, 0.00091763, 0.04820224, ..., 0.26867828, 0.40503547,
        0.0417841 ],
       [0.1432239 , 0.0009157 , 0.04811917, ..., 0.27033585, 0.40420878,
        0.04166289]], dtype=float32)

In [ ]:
"""
predicted_label = np.argmax(y_pred)
predicted_label_name = class_names[predicted_label]
print(y_pred)
print("Clase predicha:", predicted_label_name)
"""

IndexError: list index out of range

In [ ]:
predicted_label = np.argmax(y_pred)
print(predicted_label)

644


In [ ]:
y_pred[0]

array([4.2934707e-01, 1.9751508e-04, 5.6709602e-02, 4.9945887e-02,
       1.6233388e-01, 2.9200563e-01, 9.4604148e-03], dtype=float32)

In [ ]:
df_test

,imagen,path,imagenes
0,10004.jpg,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...,"[[1, 1, 1, 2, 7, 15, 16, 9, 11, 14, 13, 14, 31..."
1,10019.jpg,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...,"[[159, 157, 158, 166, 170, 170, 172, 171, 166,..."
2,10023.jpg,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...,"[[195, 198, 193, 190, 140, 70, 54, 88, 169, 21..."
3,10029.jpg,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...,"[[41, 48, 57, 66, 68, 67, 75, 85, 94, 105, 109..."
4,1003.jpg,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...,"[[44, 33, 21, 23, 29, 29, 34, 37, 40, 40, 45, ..."
...,...,...,...
7061,9977.jpg,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...,"[[178, 139, 130, 131, 159, 188, 183, 239, 249,..."
7062,9979.jpg,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...,"[[243, 243, 238, 196, 185, 176, 168, 162, 157,..."
7063,9982.jpg,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...,"[[203, 229, 234, 238, 233, 232, 238, 235, 229,..."
7064,9984.jpg,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...,"[[57, 56, 57, 63, 61, 56, 58, 58, 61, 72, 69, ..."


In [ ]:
pred_test_rounded = np.round(y_pred)

In [ ]:
pred_test_rounded

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
label_dict = {0: 'angry', 1: 'disgust', 2: "fear", 3: "happy", 4:"neutral", 5:"sad", 6:"surprise"}

pred_test_label = np.argmax(y_pred, axis=1)


In [ ]:
pred_test_label[0]

0

In [ ]:
df_predicciones = pd.DataFrame(pred_test_label, columns=['label'])

df_predicciones['label'] = df_predicciones['label'].map(label_dict)

In [ ]:
print(pred_test_rounded[0])

[0. 0. 0. 0. 0. 0. 0.]


In [ ]:
df_test["label"] = df_predicciones['label']

In [ ]:
df_test

,imagen,path,imagenes,label
0,10004,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...,"[[1, 1, 1, 2, 7, 15, 16, 9, 11, 14, 13, 14, 31...",angry
1,10019,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...,"[[159, 157, 158, 166, 170, 170, 172, 171, 166,...",angry
2,10023,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...,"[[195, 198, 193, 190, 140, 70, 54, 88, 169, 21...",angry
3,10029,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...,"[[41, 48, 57, 66, 68, 67, 75, 85, 94, 105, 109...",angry
4,1003,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...,"[[44, 33, 21, 23, 29, 29, 34, 37, 40, 40, 45, ...",angry
...,...,...,...,...
7061,9977,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...,"[[178, 139, 130, 131, 159, 188, 183, 239, 249,...",angry
7062,9979,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...,"[[243, 243, 238, 196, 185, 176, 168, 162, 157,...",angry
7063,9982,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...,"[[203, 229, 234, 238, 233, 232, 238, 235, 229,...",angry
7064,9984,c:\Users\Usuario\Desktop\Proyecto CNN alegres\...,"[[57, 56, 57, 63, 61, 56, 58, 58, 61, 72, 69, ...",angry


In [ ]:
df_submission = pd.DataFrame(df_test[["imagen", "label"]])
df_submission.to_csv("submission1.csv", index = False)